#Importing Modules

In [1]:
# Import the required modules

import random
import json
import pickle
import numpy as np
import nltk
import tensorflow as tf

In [2]:
# ONLY IF YOU ARE USING COLAB. Upload intense.json to your drive and mount the drive to access it
# Nothing to be added or changed here.

intents = json.loads(open("/content/intense.json").read())

In [3]:
from keras.models import Sequential
from nltk.stem import WordNetLemmatizer
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:

# Create a lemmatizer. This later helps in processing the input text. 
# Nothing to be added or changed here.

lemmatizer = WordNetLemmatizer()

In [6]:
# Creating empty lists to store data
# Nothing to be added or changed here.

words = []
classes = []
documents = []
ignore_letters = ["?", "!", ".", ","]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # separating words from patterns
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)  # and adding them to words list
          
        # associating patterns with respective tags
        documents.append(((word_list), intent['tag']))
  
        # appending the tags to the class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [8]:
# Store the root words or lemma using the lemmatizer
# Nothing to be added or changed here.

words = [lemmatizer.lemmatize(word)
         for word in words if word not in ignore_letters]
words = sorted(set(words))

In [9]:
# Save the words and classes list to binary files
# Nothing to be added or changed here.

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [10]:
# No changes here

training = []
output_empty = [0]*len(classes)
for document in documents:
    bag = [] # Recall Bag of Words!
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(
        word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
          
    # Make a copy of the output_empty
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)

<ipython-input-10-cd8c504277ce>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


#Training Model

In [11]:
# Split the data into train_x and train_y
train_x = list(training[:,0])
train_y = list(training[:,1])


In [12]:
# Create a Neural network using a Sequential layer, followed by a few alternating dense and dropout layers. 
# Use activation function 'relu' and a dropout of 0.5. For the last dense layer, use 'softmax' activation function

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [13]:
# Compile the model
# No changes here 

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y),
                 epochs=200, batch_size=5, verbose=1)
     

Epoch 1/200


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


4/4 [==============================] - 1s 8ms/step - loss: 1.1553 - accuracy: 0.1765
Epoch 2/200
4/4 [==============================] - 0s 8ms/step - loss: 1.0805 - accuracy: 0.2941
Epoch 3/200
4/4 [==============================] - 0s 8ms/step - loss: 1.0739 - accuracy: 0.3529
Epoch 4/200
4/4 [==============================] - 0s 8ms/step - loss: 1.0870 - accuracy: 0.4118
Epoch 5/200
4/4 [==============================] - 0s 11ms/step - loss: 1.0290 - accuracy: 0.6471
Epoch 6/200
4/4 [==============================] - 0s 8ms/step - loss: 0.9144 - accuracy: 0.5882
Epoch 7/200
4/4 [==============================] - 0s 24ms/step - loss: 1.0041 - accuracy: 0.5882
Epoch 8/200
4/4 [==============================] - 0s 14ms/step - loss: 0.9895 - accuracy: 0.5882
Epoch 9/200
4/4 [==============================] - 0s 6ms/step - loss: 0.9897 - accuracy: 0.6471
Epoch 10/200
4/4 [==============================] - 0s 6ms/step - loss: 0.8318 - accuracy: 0.7647
Epoch 11/200
4/4 [====================

In [14]:
# Save the model for use ahead
# No changes here

model.save("chatbotmodel.h5", hist)
  
# Print a message for the successful training of the Chatbot model
print("Model saved!")

Model saved!


#Executing Model

In [15]:
# No change

from keras.models import load_model

In [16]:
# Load the files we made previously - 
# load 'intense.json' into variable 'intents'
# load 'words.pk1' into variable 'words'
# load 'classes.pk1' into variable 'classes'
# load 'chatbotmodel.h5' into variable 'model'

lemmatizer = WordNetLemmatizer()
intents = json.loads(open("intense.json").read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbotmodel.h5')

In [17]:
# Create a function 'clean_up_sentences(sentence)' to separate words from the input sentences (this is called tokenisation). 
# Use nltk.word_tokenize(sentence) and return the words of the sentence after lemmatization

def clean_up_sentences(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [18]:
# Create a function 'bagw(sentence)' to  append 1 to a list variable ‘bag’ if the word is contained inside our input and is also present in the list of words created earlier.
# Return a numpy array of the list variable bag that now contains 1’s and 0’s.

def bagw(sentence):
    
    # Separate out words from the input sentence
    sentence_words = clean_up_sentences(sentence)
    bag = [0]*len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
  
            # check whether the word
            # is present in the input as well
            if word == w:
                bag[i] = 1

            
            
    # return a numpy array
    return np.array(bag)

In [19]:
# Create a function 'predict_class(sentence)' to predict the class of the sentence input by the user.

def predict_class(sentence):
  # Initialize a variable 'bow' that will contain a NumPy array of 0’s and 1’s, using the bag(w) function defined above
  # Your line of code here 
    bow = bagw(sentence)
    res = model.predict(np.array([bow]))[0]

    # In variable 'results', append from ‘res’ if the value is greater than the ERROR_THRESHOLD, then sort it using the sort function. 
    # No change
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) 
               if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]],
                            'probability': str(r[1])})
        return return_list

In [20]:
# Create 'get_response(intents_list, intents_json)' to generate a random response from whichever class the sentence/words input by the user belongs to.

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    result = ""
    for i in list_of_intents:
    # If the tag matches the tag in list_of_intents, store a random response from the 'intense.json' file in a variable called 'result'
    # your code here
        if i['tag'] == tag:
            result = random.choice(i['responses'])  
            break

        
    return result
  
print("Chatbot is up!")

Chatbot is up!


#Implementing Chatbot 

In [31]:
# Create an infinite loop to keep the model running.
# No edits here

while True:
    message = input("")
    ints = predict_class(message)
    print(ints)
    res = get_response(ints, intents)
    print(res)
    # Write a couple of lines of code to stop the infinite loop if the intent of the message is 'goodbye'
    if ints[0]['intent'] == "goodbye":
      break

hi
1/1 [==============================] - 0s 21ms/step
[{'intent': 'greeting', 'probability': '0.74599296'}]
Hi there, how can I help?
thanks
1/1 [==============================] - 0s 24ms/step
[{'intent': 'thanks', 'probability': '0.99995923'}]
My pleasure
hello again
1/1 [==============================] - 0s 21ms/step
[{'intent': 'greeting', 'probability': '0.74599296'}]
Good to see you again
meet again
1/1 [==============================] - 0s 22ms/step
[{'intent': 'greeting', 'probability': '0.74599296'}]
Good to see you again
bye
1/1 [==============================] - 0s 18ms/step
[{'intent': 'goodbye', 'probability': '0.99795246'}]
Bye! Come back again soon.
